In [1]:
from selenium import webdriver
import base64, pickle, time, os
from IPython.display import Image
from IPython.display import display
from PIL import Image as pil
from selenium.webdriver.chrome.options import Options

### 1. Get Youtube Images

##### (1) Get Youtube Driver & Save Element Screen

In [38]:
def getDriver(youtube_url):
    # size를 정의하지 않으면 chrome이 현재 화면의 반 정도의 size로 실행되고 그 상태에서의 좌표는 실제 youtube movie player 화면과 다름
    # 자신의 해상도에 맞게 size를 정의한다 
    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(chrome_options=chrome_options)
    driver.get(youtube_url)
    return driver

def saveScreen(driver, filename):
    element = driver.find_element_by_css_selector('#movie_player')
    location = element.location
    size = element.size
    driver.save_screenshot('data/images/screenshot.png')
    im = pil.open('data/images/screenshot.png')
    
    left = location['x']
    top = location['y']
    right = left + size['width']
    bottom = top + size['height']

    # left, top, right, bottom
    im = im.crop((left, top, right, bottom)) 
    im.save(filename) 

##### (2) Check Element

In [15]:
def checkElement(driver, selector):
    result = True
    try:
        driver.find_element_by_css_selector(selector)
    except:
        result = False
    return result

##### (3) Skip Ad

In [16]:
skipAd_selector = ".videoAdUiSkipButton.videoAdUiAction.videoAdUiFixedPaddingSkipButton"
closeAd_selector = ".close-padding .close-button"

def skipAd(driver):
    if checkElement(driver, skipAd_selector):
        driver.find_element_by_css_selector(skipAd_selector).click()
    
def closeAd(driver):
    if checkElement(driver, closeAd_selector):
        driver.find_element_by_css_selector(closeAd_selector).click()

##### (4) Make Directory

In [17]:
def makeDir(youtube_url):
    dirList = os.listdir("data/images/")
    dirName = youtube_url.split("v=")[1]
    isDir = True
    if dirName not in dirList:
        os.makedirs("data/images/" + dirName)
        isDir = False
    return isDir, dirName

### 2. Google Vision API

In [18]:
# single thread
def vision(filenames):
    url = 'https://cloud.google.com/vision/'

    # open site
    driver =  webdriver.Chrome()
    driver.get(url)

    # enter iframe
    iframe = driver.find_element_by_css_selector("#vision_demo_section iframe")
    driver.switch_to_frame(iframe)
    
    result = []
    
    for filename in filenames:
    
        # file upload
        driver.find_element_by_id("input").send_keys(filename)

        # check done analytics
        delay = 0
        for _ in range(10):
            if driver.find_element_by_css_selector("#results").text != '':
                break
            time.sleep(1)
            delay += 1
        # print("Analytics Delay Time : {} sec".format(delay))
        
        if delay >= 10:
            continue
        
        # check face

        isFace = True
        time.sleep(1)
        try:
            driver.find_element_by_css_selector('.face.style-scope.vs-faces')
        except:
            isFace = False

        face_data_list = []

        if isFace:
            face_list = driver.find_elements_by_css_selector('.face.style-scope.vs-faces')
            for idx, face in enumerate(face_list):
                faceData = {
                    "name":"face " + str(idx+1),
                    "confidence": face.find_element_by_css_selector('.confidence .conf-score.style-scope.vs-faces').text
                }          
                expressions = face.find_elements_by_css_selector('.type-row.style-scope.vs-faces')
                expressionData = {}
                for expression in expressions:
                    name = expression.find_element_by_css_selector('#label').text
                    state = expression.find_element_by_css_selector('#text').text
                    if "Likely" in state or "Possible" in state or "Very Likely" in state:
                        state = 1
                    else:
                        state = 0
                    expressionData[name] = state
                faceData["expressions"] = expressionData
                face_data_list.append(faceData)  
                
        result.append(face_data_list)    
        
        # close image
        driver.find_element_by_css_selector("#exit").click()
    driver.close()
    return result

In [19]:
def saveImages(youtube_url, start_term, image_count, save_term, dirName):
    
    # open driver
    driver = getDriver(youtube_url)

    # skip ad
    time.sleep(6)
    skipAd(driver)
    time.sleep(6)
    closeAd(driver)

    # save images
    time.sleep(start_term)
    for idx in range(1, image_count+1):
        saveScreen(driver, 'data/images/' + dirName + '/screenshot' + str(idx) + '.png')
        time.sleep(save_term)
        print('saved : screenshot' + str(idx) + '.png')
    # close driver
    driver.close()

In [20]:
def makeTunmnail(image_count, dirName):
# make image list
    filenames = []
    for idx in range(1, image_count + 1):
        filename = 'D:/101. source/study/python/data/images/' + dirName  + '/screenshot' + str(idx) + '.png'
        filenames.append(filename)

    # make thumnail
    size = (512, 512)
    thumnails = []
    for filename in filenames:
        im = pil.open(filename)
        im.thumbnail(size)
        savename = filename.split("/")
        savename[len(savename)-1] = savename[len(savename)-1].replace(".png","_thumbnail.png")
        savename = "/".join(savename)
        im.save(savename, "png")
        thumnails.append(savename)
        
    return thumnails

In [39]:
youtube_url = "https://www.youtube.com/watch?v=xyq2T_K6DHg&start=110" # 이미지 생성 youtube URL 
#youtube_url = "https://www.youtube.com/watch?v=gQ99oQrvIf4"
#youtube_url = "https://www.youtube.com/watch?v=vj_mEaRez-E"

# 뿜뿜 
#youtube_url = "https://www.youtube.com/watch?v=hLv5MBuSYDY&start=60"
#youtube_url = "https://www.youtube.com/watch?v=hLv5MBuSYDY"

In [40]:
%%time
isDir, dirName = makeDir(youtube_url)
start_term = 5 # 5초 후 부터 이미지 생성
image_count = 5 # 5개의 이미지 생성
save_term = 0.3 # 저장 텀
saveImages(youtube_url, start_term, image_count, save_term, dirName)
# make thumnail
thumnails = makeTunmnail(image_count, dirName)

saved : screenshot1.png
saved : screenshot2.png
saved : screenshot3.png
saved : screenshot4.png
saved : screenshot5.png
Wall time: 31.7 s


In [41]:
%%time
result = vision(thumnails)

Wall time: 36.5 s


In [42]:
expression_list = ['Joy', 'Sorrow', 'Anger', 'Surprise', 'Exposed', 'Blurred', 'Headwear']
person_count = 0
scene_count = 0

expressions = {}
for expression in expression_list:
    expressions[expression] = 0

for data in result:
    scene_count += 1
    person_count += len(data)
    for face in data:
        confidence = float(face["confidence"].replace("%", "")) * 0.01
        confidence = round(confidence, 2)
        for expression in expression_list:
            number = int(face['expressions'][expression]) * confidence / person_count
            expressions[expression] = round(expressions[expression] + number, 2)

In [43]:
summary_data = {
    "scene_count":scene_count,
    "person_count":person_count,
    "expressions":expressions,
}

# save
path = 'data/images/' + dirName  + '/result.plk'
pickle.dump(summary_data, open(path, 'wb'))

# read
with open(path, 'rb') as file:
    summary_data = pickle.load(file)
    print(summary_data)

{'scene_count': 5, 'person_count': 15, 'expressions': {'Joy': 0.23, 'Sorrow': 0.0, 'Anger': 0.0, 'Surprise': 0.0, 'Exposed': 0.0, 'Blurred': 0.0, 'Headwear': 0.2}}
